In [8]:
import pandas as pd
import torch
import copy
import os

from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.metrics import r2_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
#import dataset
df = pd.read_csv('dataset-ml-25m/dataset.csv')

In [11]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
#count the numebr of x_train 
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])


Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [13]:
batch_sizes = [8, 16, 32]
n_epochs = [200]
# Dimension of the prediction  layer
n_d = [8, 16, 32]
#Dimension of the attention  layer
n_a = [8, 16, 32]
#Number of successive steps in the network
n_steps = [3, 5, 7]
# Number of independent GLU layer in each GLU block
n_indipendent = [2, 3]

params = list(product(batch_sizes, n_epochs, n_d, n_a, n_steps, n_indipendent))
comb = len(batch_sizes)*len(n_d)*len(n_a) *len(n_steps) *len(n_indipendent)
print("Number of combinations: ", comb)

Number of combinations:  162


In [14]:
def get_model(batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent):
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps,
        n_independent=n_indipendent
    )
    return model

In [15]:
best_mse = float('inf')
best_model = None
best_params = None
iter = 0
for b, n_e, n_d, n_a, n_s, n_i in params:
    iter += 1
    print(f'\nIteration {iter}/{comb}')
    print(f"Configuration batch size: {b} - epochs: {n_e} - n_d: {n_d} - n_a: {n_a} - steps: {n_s} - n_indipendent: {n_i}")
    
    model = get_model(b, n_e, n_d, n_a, n_s, n_i)
    #save results for each iteration with tensorboard
    log = f'bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}'

    if os.path.exists("./results/TabNet"+log):
        print("Model already trained. Skipping...")
        continue

    writer = SummaryWriter(f'results/TabNet/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')
    #fit model
    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['mse'],
        patience=10,
        batch_size=b
    )

    # evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
   #save hparams for each iteration with tensorboard
    writer.add_hparams(
        {'batch_size': b, 'n_epochs': n_e, 'n_d': n_d, 'n_a': n_a, 'n_steps': n_s, 'n_indipendent': n_i},
        {'hparam/mse': mse, 'hparam/r2': r2}
    )
    print('MSE:', mse)
    print('R2 Score:', r2)
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_params = (b, n_e, n_d, n_a, n_s, n_i)
        print('Best model updated')
    writer.close()
            


Iteration 1/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 2


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.2812  | mse_mse: 0.07043 |  0:00:22s
epoch 1  | loss: 0.09894 | mse_mse: 0.27305 |  0:00:45s
epoch 2  | loss: 0.0819  | mse_mse: 0.07393 |  0:01:12s
epoch 3  | loss: 0.07828 | mse_mse: 0.06278 |  0:01:37s
epoch 4  | loss: 0.07167 | mse_mse: 0.05142 |  0:02:04s
epoch 5  | loss: 0.07409 | mse_mse: 0.06293 |  0:02:27s
epoch 6  | loss: 0.07251 | mse_mse: 0.06103 |  0:02:49s
epoch 7  | loss: 0.06916 | mse_mse: 0.0606  |  0:03:11s
epoch 8  | loss: 0.06491 | mse_mse: 0.0412  |  0:03:34s
epoch 9  | loss: 0.0633  | mse_mse: 0.05938 |  0:03:57s
epoch 10 | loss: 0.06093 | mse_mse: 0.10407 |  0:04:20s
epoch 11 | loss: 0.06202 | mse_mse: 0.06393 |  0:04:42s
epoch 12 | loss: 0.0585  | mse_mse: 0.07598 |  0:05:05s
epoch 13 | loss: 0.05858 | mse_mse: 0.07784 |  0:05:27s
epoch 14 | loss: 0.06011 | mse_mse: 0.04215 |  0:05:49s
epoch 15 | loss: 0.05708 | mse_mse: 0.04663 |  0:06:11s
epoch 16 | loss: 0.05288 | mse_mse: 0.0658  |  0:06:35s
epoch 17 | loss: 0.04876 | mse_mse: 0.05388 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.03906676625632957
R2 Score: 0.8237584937093532
Best model updated

Iteration 2/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 3


...

MSE: 0.014322879234792673
R2 Score: 0.9353853402097261

Iteration 162/162
Configuration batch size: 32 - epochs: 200 - n_d: 32 - n_a: 32 - steps: 7 - n_indipendent: 3


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.40152 | mse_mse: 0.22092 |  0:00:16s
epoch 1  | loss: 0.16732 | mse_mse: 0.14049 |  0:00:34s
epoch 2  | loss: 0.12125 | mse_mse: 0.10973 |  0:00:52s
epoch 3  | loss: 0.11088 | mse_mse: 0.08721 |  0:01:10s
epoch 4  | loss: 0.09843 | mse_mse: 0.05674 |  0:01:28s
epoch 5  | loss: 0.08492 | mse_mse: 0.07692 |  0:01:46s
epoch 6  | loss: 0.07659 | mse_mse: 0.05528 |  0:02:05s
epoch 7  | loss: 0.05442 | mse_mse: 0.0351  |  0:02:23s
epoch 8  | loss: 0.04959 | mse_mse: 0.03067 |  0:02:42s
epoch 9  | loss: 0.03579 | mse_mse: 0.0405  |  0:03:00s
epoch 10 | loss: 0.03607 | mse_mse: 0.03656 |  0:03:18s
epoch 11 | loss: 0.03079 | mse_mse: 0.06472 |  0:03:37s
epoch 12 | loss: 0.02575 | mse_mse: 0.011   |  0:03:55s
epoch 13 | loss: 0.02556 | mse_mse: 0.03042 |  0:04:14s
epoch 14 | loss: 0.03074 | mse_mse: 0.01628 |  0:04:32s
epoch 15 | loss: 0.02534 | mse_mse: 0.01868 |  0:04:50s
epoch 16 | loss: 0.02595 | mse_mse: 0.04532 |  0:05:09s
epoch 17 | loss: 0.02925 | mse_mse: 0.01111 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.011844174563179278
R2 Score: 0.9465674954490026


In [16]:
#pint the best model
print(best_model)
print("Best MSE: ", best_mse)
print("R2 score", r2_score(y_test, best_model.predict(X_test)))


TabNetRegressor(n_d=16, n_a=32, n_steps=7, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=1128, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)
Best MSE:  0.00639599107305163
R2 score 0.9711458303577016


In [17]:
#save the best model in a file csv
best_model.save_model('best_model_tabnet.csv')

Successfully saved model at best_model_tabnet.csv.zip


'best_model_tabnet.csv.zip'